## Data Types ##
```
Name            | Number of bytes | Description
--------------------------------------------------------------------
Int             |               4 | integer
String          |        Variable | (See Note 1)
Boolean         |               1 | 0x00 for false, everything else is true
DateTime        |               8 | (See Note 2)
Byte            |               1 | integer
Short           |               2 | integer
Single          |               4 | 32-bit IEEE floating point value
Double          |               8 | 64-bit IEEE floating point value
Int-Double pair |              14 | (See Note 3)
Timing point    |              17 | (See Note 4)
```

Note 1: String has three parts; a single byte which will be either 0x00, indicating that the next two parts are not present, or 0x0b (decimal 11), indicating that the next two parts are present. If it is 0x0b, there will then be a ULEB128, representing the byte length of the following string, and then the string itself, encoded in UTF-8. See this.

Note 2: A 64-bit number of ticks representing a date and time. Ticks are the amount of 100-nanosecond intervals since midnight, January 1, 0001 UTC. See .NET framework documentation on ticks for more information.

Note 3: The first byte is 0x08, followed by an Int, then 0x0d, followed by a Double. These extraneous bytes are presumably flags to signify different data types in these slots, though in practice no other such flags have been seen. Currently the purpose of this data type is unknown.

Note 4: Consists of a Double, signifying the BPM, another Double, signifying the offset into the song, in milliseconds, and a Boolean; if false, then this timing point is inherited. See [Osu (file format)][Osu Link] for more information regarding timing points.

## osu!.db format ##
### Header ###
```
Data type | Description
----------------------------------------------------------------------------------------
Int       | osu! version (e.g. 20150203)
Int       | Folder Count
Bool      | AccountUnlocked (only false when the account is locked or banned in any way)
DateTime  | Date the account will be unlocked
String    | Player name
Int       | Number of beatmaps
Beatmaps* | (See Beatmap below)
Int       | Unknown Int, always seems to be 4
```
### Beatmap ###
```
Data type        | Description
----------------------------------------------------------------------------------------
Int              | Size in bytes of the beatmap entry
String           | Artist name
String           | Artist name, in Unicode
String           | Song title
String           | Song title, in Unicode
String           | Creator name
String           | Difficulty (e.g. Hard, Insane, etc.)
String           | Audio file name
String           | MD5 hash of the beatmap
String           | Name of the .osu file corresponding to this beatmap
Byte             | Ranked status (0 = unknown, 1 = unsubmitted, 2 = pending/wip/graveyard, 3 = unused, 4 = ranked, 5 = approved, 6 = qualified, 7 = loved)
Short            | Number of hitcircles
Short            | Number of sliders (note: this will be present in every mode)
Short            | Number of spinners (note: this will be present in every mode)
Long             | Last modification time, Windows ticks.
Byte/Single      | Approach rate. Byte if the version is less than 20140609, Single otherwise.
Byte/Single      | Circle size. Byte if the version is less than 20140609, Single otherwise.
Byte/Single      | HP drain. Byte if the version is less than 20140609, Single otherwise.
Byte/Single      | Overall difficulty. Byte if the version is less than 20140609, Single otherwise.
Double           | Slider velocity
Int-Double pair* | An Int indicating the number of following Int-Double pairs, then the aforementioned pairs. Star Rating info for osu! standard, in each pair, the Int is the mod combination, and the Double is the Star Rating. Only present if version is greater than or equal to 20140609.
Int-Double pair* | An Int indicating the number of following Int-Double pairs, then the aforementioned pairs. Star Rating info for Taiko, in each pair, the Int is the mod combination, and the Double is the Star Rating. Only present if version is greater than or equal to 20140609.
Int-Double pair* | An Int indicating the number of following Int-Double pairs, then the aforementioned pairs. Star Rating info for CTB, in each pair, the Int is the mod combination, and the Double is the Star Rating. Only present if version is greater than or equal to 20140609.
Int-Double pair* | An Int indicating the number of following Int-Double pairs, then the aforementioned pairs. Star Rating info for osu!mania, in each pair, the Int is the mod combination, and the Double is the Star Rating. Only present if version is greater than or equal to 20140609.
Int              | Drain time, in seconds
Int              | Total time, in milliseconds
Int              | Time when the audio preview when hovering over a beatmap in beatmap select starts, in milliseconds.
Timing point+    | An Int indicating the number of following Timing points, then the aforementioned Timing points.
Int              | Beatmap ID
Int              | Beatmap set ID
Int              | Thread ID
Byte             | Grade achieved in osu! standard.
Byte             | Grade achieved in Taiko.
Byte             | Grade achieved in CTB.
Byte             | Grade achieved in osu!mania.
Short            | Local beatmap offset
Single           | Stack leniency
Byte             | Osu gameplay mode. 0x00 = osu!Standard, 0x01 = Taiko, 0x02 = CTB, 0x03 = Mania
String           | Song source
String           | Song tags
Short            | Online offset
String           | Font used for the title of the song
Boolean          | Is beatmap unplayed
Long             | Last time when beatmap was played
Boolean          | Is the beatmap osz2
String           | Folder name of the beatmap, relative to Songs folder
Long             | Last time when beatmap was checked against osu! repository
Boolean          | Ignore beatmap sound
Boolean          | Ignore beatmap skin
Boolean          | Disable storyboard
Boolean          | Disable video
Boolean          | Visual override
Short?           | Unknown. Only present if version is less than 20140609.
Int              | Last modification time (?)
Byte             | Mania scroll speed
```

In [ ]:
import sys
sys.path.append("projects/osu!tools/")

In [ ]:
from db import *

In [ ]:
from pathlib import Path
user_home = str(Path.home())
osu_osu_file = user_home + '\\AppData\\Local\\osu!\\osu!.db'

In [ ]:
OSU_GAME_TYPES = ['standard', 'taiko', 'ctb', 'mania']
OSU_BEATMAP_RANKED_STATUS = ['unknown', 'unsubmitted', 'pending/wip/graveyard', 'unused','ranked', 'approved', 'qualified', 'loved']

with open(osu_osu_file, "rb") as f:
    
    osu_version, folder_count = read_int(f), read_int(f)
    print('osu! version:', osu_version)
    print('Folder Count:', folder_count)
    
    account_unlocked, date_account_unlocked = read_bool(f), read_datetime(f)
    print('AccountUnlocked:', account_unlocked)
    print('Date the account will be unlocked:', date_account_unlocked)
    
    player_name = read_string(f)
    print('Player name:', player_name)
    
    beatmap_count = read_int(f)
    print('Number of beatmaps:', beatmap_count)
    
    # beatmap
    beatmap_count = 5
    for c in range(beatmap_count):
    
        print('-'*20, 'beatmap', c+1, '-'*20)
        
        entry_size = read_int(f)
        print('beatmap entry size (bytes):', entry_size)
        
        artist_name, artist_name_u = read_string(f), read_string(f)
        print('Artist name:', artist_name)
        print('Artist name (Unicode):', artist_name_u)
        
        song_title, song_title_u = read_string(f), read_string(f)
        print('Song title:', song_title)
        print('Song title (Unicode):', song_title_u)
        
        creator_name = read_string(f)
        print('Creator name:', creator_name)
        
        difficulty = read_string(f)
        print('Difficulty:', difficulty)
        
        audio_file_name = read_string(f)
        print('Audio file name:', audio_file_name)
        
        beatmap_hash = read_string(f)
        print('MD5 hash of the beatmap:', beatmap_hash)
        
        osu_filename = read_string(f)
        print('.osu filename:', osu_filename)
        
        ranked_status = read_byte(f)
        print('Ranked status:', ranked_status, '=', OSU_BEATMAP_RANKED_STATUS[ranked_status])
        
        hitcircle_count, slider_count, spinner_count = read_short(f), read_short(f), read_short(f)
        print('Number of hitcircles:', hitcircle_count)
        print('Number of sliders:', slider_count)
        print('Number of spinners:', spinner_count)
        
        last_mod_time = read_datetime(f)
        print('Last modification time:', last_mod_time)
        
        approach_rate, circle_size, hp_drain = read_single(f), read_single(f), read_single(f)
        print('Approach rate:', approach_rate)
        print('Circle size:', circle_size)
        print('HP drain:', hp_drain)
        
        overall_difficulty = read_single(f)
        print('Overall difficulty:', overall_difficulty)
        
        slider_velocity = read_double(f)
        print('Slider velocity:', slider_velocity)
        
        star_ratings = {}
        for game_type in OSU_GAME_TYPES:
            pair_count = read_int(f)
            
            mod_stars = []
            for _ in range(pair_count):
                read_byte(f)  # 0x08
                mod_comb = read_int(f)
                read_byte(f)  # 0x0d
                stars = read_double(f)
                mod_stars.append((mod_comb, stars))
                
            star_ratings[game_type] = mod_stars
        print('Star Ratings:', star_ratings)
        
        drain_time, total_time, preview_time = read_int(f), read_int(f), read_int(f)
        print('Drain time(s):', drain_time)
        print('Total time(ms):', total_time)
        print('Preview time(ms):', preview_time)
        
        timing_point_count = read_int(f)
        timing_points = []
        for _ in range(timing_point_count):
            bpm, offset, inherited = read_double(f), read_double(f), not read_bool(f)
            timing_points.append((bpm, offset, inherited))
        print('Timing points:', timing_points)
        
        beatmap_id, beatmap_set_id, thread_id = read_int(f), read_int(f), read_int(f)
        print('Beatmap ID:', beatmap_id)
        print('Beatmap set ID:', beatmap_set_id)
        print('Thread ID:', thread_id)
        
        grades = {game_type: read_byte(f) for game_type in OSU_GAME_TYPES}
        print('Grades:', grades)
        
        local_beatmap_offset = read_short(f)
        print('Local beatmap offset:', local_beatmap_offset)
        
        stack_leniency = read_single(f)
        print('Stack leniency:', stack_leniency)
        
        # gameplay mode: 0x00 = osu!Standard, 0x01 = Taiko, 0x02 = CTB, 0x03 = Mania
        gameplay_mode = read_byte(f)
        print('Osu gameplay mode:', gameplay_mode, '=', OSU_GAME_TYPES[gameplay_mode])
        
        song_source, song_tags = read_string(f), read_string(f)
        print('Song source:', song_source)
        print('Song tags:', song_tags)
        
        online_offset = read_short(f)
        print('Online offset:', online_offset)

        song_font = read_string(f)
        print('Font used for the title of the song:', song_font)

        beatmap_unplayed = read_bool(f)
        print('Is beatmap unplayed:', beatmap_unplayed)

        beatmap_last_played = read_datetime(f)
        print('Last time when beatmap was played:', beatmap_last_played)

        beatmap_osz2 = read_bool(f)
        print('Is the beatmap osz2:', beatmap_osz2)

        beatmap_folder = read_string(f)
        print('Folder name of the beatmap, relative to Songs folder:', beatmap_folder)

        beatmap_last_checked = read_datetime(f)#long(f)
        print('Last time when beatmap was checked against osu! repository:', beatmap_last_checked)
        
        ignore_sound, ignore_skin, disable_storyboard, disable_video, visual_override = \
            read_bool(f), read_bool(f), read_bool(f), read_bool(f), read_bool(f)
        print('Ignore beatmap sound:', ignore_sound)
        print('Ignore beatmap skin:', ignore_skin)
        print('Disable storyboard:', ignore_skin)
        print('Disable video:', ignore_skin)
        print('Visual override:', visual_override)
        
        last_modified = read_int(f)
        print('Last modification time:', last_modified)
        
        mania_scroll_speed = read_byte(f)
        print('Mania scroll speed:', mania_scroll_speed)
        
#             print(f'beatmap#{b+1} MD5: {beatmap_md5}')